# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/kaijagregory/Desktop/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,288.02,67,40,10.80,PT,1701330432
1,1,vilyuchinsk,52.9306,158.4028,269.30,77,100,4.70,RU,1701330432
2,2,adamstown,-25.0660,-130.1015,294.33,77,100,3.73,PN,1701330432
3,3,palmer,42.1584,-72.3287,272.86,60,100,2.57,US,1701330405
4,4,mubende,0.5606,31.3900,299.90,41,56,2.49,UG,1701330432


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    frame_width=700,
    frame_height=400
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria
max_temp_criteria = city_data_df["Max Temp"] <= 305
humidity_criteria = city_data_df["Humidity"] <= 50
wind_speed_criteria = city_data_df["Wind Speed"] < 10

ideal_cities_df = city_data_df.loc[max_temp_criteria & humidity_criteria & wind_speed_criteria]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,mubende,0.5606,31.3900,299.90,41,56,2.49,UG,1701330432
26,26,zabol,32.2500,67.2500,286.05,26,0,6.09,AF,1701330436
28,28,nyala,12.0500,24.8833,302.38,17,90,8.70,SD,1701330437
29,29,fereydunshahr,32.9410,50.1210,283.04,13,0,0.85,IR,1701330437
35,35,sayat,38.7839,63.8803,280.08,32,13,2.95,TM,1701330438


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the 
# city, country, coordinates, and humidity
# Use the Pandas copy function to create a new DataFrame (hotel_df)
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name" to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,mubende,UG,0.5606,31.3900,41,
26,zabol,AF,32.2500,67.2500,26,
28,nyala,SD,12.0500,24.8833,17,
29,fereydunshahr,IR,32.9410,50.1210,13,
35,sayat,TM,38.7839,63.8803,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 20,
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mubende - nearest hotel: Hotel Nakayima
zabol - nearest hotel: No hotel found
nyala - nearest hotel: سوني سفاري
fereydunshahr - nearest hotel: No hotel found
sayat - nearest hotel: No hotel found
bandiagara - nearest hotel: Hôtel Auberge Kansaye
dubbo - nearest hotel: Blue Gum Motel
tura - nearest hotel: No hotel found
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
umm kaddadah - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn
rawson - nearest hotel: Hotel Deportivo
hermanus - nearest hotel: Aloe guest house
zhenzhou - nearest hotel: 白沙宾馆
jinchang - nearest hotel: No hotel found
birao - nearest hotel: No hotel found
bhawanipatna - nearest hotel: Hotel Centre Park
timimoun - nearest hotel: أناف
taft - nearest hotel: هتل شیرکوه تفت
keshorai patan - nearest hotel: No hotel found
dakoutun - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
waddan - nearest hotel: No hotel found
dwarka - nearest hotel: The Dwari

,City,Country,Lat,Lng,Humidity,Hotel Name
4,mubende,UG,0.5606,31.3900,41,Hotel Nakayima
26,zabol,AF,32.2500,67.2500,26,No hotel found
28,nyala,SD,12.0500,24.8833,17,سوني سفاري
29,fereydunshahr,IR,32.9410,50.1210,13,No hotel found
35,sayat,TM,38.7839,63.8803,32,No hotel found
...,...,...,...,...,...,...
548,taoudenni,ML,22.6783,-3.9836,15,No hotel found
554,el bauga,SD,18.2620,33.9081,39,No hotel found
556,shache,CN,38.4167,77.2406,20,莎車新盛賓舘
563,gewane,ET,10.1664,40.6453,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    frame_width=700,
    frame_height=400,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)